# API 
2025 labs

original by Grandmaster-reviewer Дуркин Анатолий Альбертович

In [6]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import requests
import json

## DS 5

In [36]:
url = 'https://api.hh.ru/vacancies'
params = {'text': 'Data Scientist', 'per_page': '5', 'page': '1'}
src = requests.get(url, params=params)
data = src.json()

Мы указали адрес, откуда получать данные. Здесь взаимодействие происходит не через библиотеку или функции, а при обращении к определенному адресу.

Также мы указываем параметры: должность, которую ищем, сколько вакансий на странице отображать и какую страницу читаем.

А дальше мы просто получаем данные по указанным параметрам и превращаем их в удобный нам формат. В данном случае данные возвращаются нам в формате JSON - очень популярном для общения сервисов в сети - поэтому в таком виде их и обрабатываем.

Посмотрим, что мы получили.

In [37]:
data

{'items': [{'id': '128921862',
   'premium': False,
   'name': 'Project Manager (AI)',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '1002',
    'name': 'Минск',
    'url': 'https://api.hh.ru/areas/1002'},
   'salary': {'from': 1118, 'to': 1615, 'currency': 'USD', 'gross': False},
   'salary_range': {'from': 1118,
    'to': 1615,
    'currency': 'USD',
    'gross': False,
    'mode': {'id': 'MONTH', 'name': 'За\xa0месяц'},
    'frequency': {'id': 'TWICE_PER_MONTH', 'name': 'Два раза в\xa0месяц'}},
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': {'city': 'Минск',
    'street': 'Национальный аэропорт Минск',
    'building': None,
    'lat': 53.889263,
    'lng': 28.034156,
    'description': None,
    'raw': 'Минск, Национальный аэропорт Минск',
    'metro': None,
    'metro_stations': [],
    'id': '13721893'},
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2025-12-19T17:50:24+0300',
   'cr

Мы получили вакансии, как и запрашивали. Вот только теперь они лежат в словарях, которые нам предстоит разобрать. Описанипе всех тегов вы можете найти в документации.

Итак, посмотрим, какие ключи есть в нашем словаре.

In [38]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

Здесь указаны параметры, которые мы передавали (относительно страниц и числа вакансий на них), а также прочие основные параметры. Но нас интересует только список с вакансиями, он лежит с ключом `items`.

In [39]:
len(data['items'])

5

Тут мы оценили размер списка, лежащего под указанным ключом. В нём действительно 20 элементов, как мы и запрашивали. Посмотрим на первый.

In [40]:
data['items'][0]

{'id': '128921862',
 'premium': False,
 'name': 'Project Manager (AI)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1002',
  'name': 'Минск',
  'url': 'https://api.hh.ru/areas/1002'},
 'salary': {'from': 1118, 'to': 1615, 'currency': 'USD', 'gross': False},
 'salary_range': {'from': 1118,
  'to': 1615,
  'currency': 'USD',
  'gross': False,
  'mode': {'id': 'MONTH', 'name': 'За\xa0месяц'},
  'frequency': {'id': 'TWICE_PER_MONTH', 'name': 'Два раза в\xa0месяц'}},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Минск',
  'street': 'Национальный аэропорт Минск',
  'building': None,
  'lat': 53.889263,
  'lng': 28.034156,
  'description': None,
  'raw': 'Минск, Национальный аэропорт Минск',
  'metro': None,
  'metro_stations': [],
  'id': '13721893'},
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2025-12-19T17:50:24+0300',
 'created_at': '2025-12-19T17:50:24+0300',
 'archived': False,
 'apply_alte

## Параметры

Мы видим большое количество параметров, свойств, описание которых (подробное и на русском) есть в документации. Отсюда многое можно взять в нашу таблицу для анализа. Я возьму лишь пару параметров для демонстрации.

In [41]:
vacancy = data['items'][0]

In [42]:
vacancy['name']

'Project Manager (AI)'

In [43]:
vacancy['id']

'128921862'

In [44]:
vacancy['salary']

{'from': 1118, 'to': 1615, 'currency': 'USD', 'gross': False}

Вот тут могут начинаться первые проблемы. Данных о зарплате может и не быть вовсе, а могут быть неполные. Тут стоит обрабатывать данные аккуратно.

Подберём две вакансии, одну с зарплатой, другую без, и посмотрим разницу.

In [45]:
# с зарплатой
vacancy_s = [v for v in data['items'] if v['salary']][0]
# без зарплаты
vacancy_w = [v for v in data['items'] if not v['salary']][0]

Дадите объяснение моим действиям? Если не очень понятно, то пустое значение `None` приравнивается к `False`, поэтому условие выглядит именно так. А как бы сделали вы?

In [46]:
vacancy_w['salary']

Тут и правда не указаны зарплаты. Что ж, посмотрим на вторую вакансию.

In [47]:
vacancy_s['salary']

{'from': 1118, 'to': 1615, 'currency': 'USD', 'gross': False}

Внутри не просто число, внутри лежит целый словарь (если что-то пошло не так, выгрузите список вакансий снова или увеличьте размер выгрузки, возможно, в этом нет вакансий с зарплатой). А в нём указаны границы зарплатной вилки, валюта и даже указание, до или после вычета налогов.

Как бы вы поступили с такими данными? Я возьму только одно из значений - нижнюю границу зарплаты. Но это для демонстрации, ваше решение явно будет отличаться.

In [48]:
vacancy_s['salary']['from']

1118

Это всё хорошо, но как же достать это значение, если этого словаря не будет вообще? Конечно, проверить на наличие. Давайте напишем небольшой цикл, пробежав по имеющимся у нас вакансиям.

In [49]:
for vacancy in data['items']:
    if vacancy['salary']:
        print(vacancy['salary']['from'])
    else:
        print('Не указано')

1118
Не указано
Не указано
1400000
Не указано


Сможем ли мы сократить это условие? Конечно, воспользуемся тернарным оператором:

In [50]:
for vacancy in data['items']:
    print(vacancy['salary']['from'] if vacancy['salary'] else 'Не указано')

1118
Не указано
Не указано
1400000
Не указано


Прекрасно! А главное, кратко и понятно.

## Собираем таблицу

Попробуем создать небольшую таблицу для данных. Я создам совсем небольшую, чтобы посмотреть, как это работает.

In [51]:
df = pd.DataFrame(columns=['name', 'salary_from'])

for vacancy in data['items']:
    id = vacancy['id']
    name = vacancy['name']
    salary_from = vacancy['salary']['from'] if vacancy['salary'] else None
    df.loc[id] = [name, salary_from]

In [52]:
df

,name,salary_from
128921862,Project Manager (AI),1118
128731045,Аналитик больших данных/Data Scientist,None
128544099,Data Scientist,None
128586832,Middle Data Scientist,1400000
128147364,Исследователь/Research Scientist,None


Да, табличка и правда получилась!

Конечно, я оставил для вас самое интересное. Как минимум, примените бритву Оккама - тут слишком много лишних сущностей, можно и короче. А как максимум...

## Парсим интересные вакансии

Как максимум я хочу, чтобы вы собрали большую таблицу по вакансиям и проанализировали её. При этом выберите ту должность, которая интересно лично вам.

Предлагаю такие шаги:

1. Какие данные из вакансии заслуживают внимания? Что стоит доставать и помещать в таблицу? Уделите особое внимание численным параметрам - с остальными мы пока не особо учились работать.
2. Какие данные могут быть проблемными? Где-то могут встречаться пропуски, как в зарплатах, их нужно обработать.
3. Как собрать больше вакансий? Больше 100 на странице не получится, а страниц может быть несколько. Надо собрать данные со всех.
4. Какой базовый анализ можно провести? Интересно, какие зарплаты предлагают по должностям, в каких городах сколько вакансий, и прочее, прочее, прочее...

Уверен, это не составит большого труда, у вас всё получится!

In [56]:
import requests, time

def get_vacancies_minimal(text="Data Scientist"):
    url = 'https://api.hh.ru/vacancies'
    results = []
    
    for page in range(5):
        response = requests.get(url, params={
            'text': text,
            'per_page': 100,
            'page': page
        }, headers={'User-Agent': 'TestBot'})
        
        if response.status_code == 200:
            data = response.json()
            if data.get('items'):
                results.extend(data['items'])
                print(f"Page {page}: {len(data['items'])} items")
            else:
                break
        time.sleep(0.3)
    
    print(f"Total: {len(results)} vacancies")
    return results

In [57]:
my_data = get_vacancies_minimal(text="web")

Page 0: 100 items
Page 1: 100 items
Page 2: 100 items
Page 3: 100 items
Page 4: 100 items
Total: 500 vacancies


[{'id': '128818364',
  'premium': False,
  'name': 'Веб разработчик',
  'department': None,
  'has_test': False,
  'response_letter_required': False,
  'area': {'id': '159',
   'name': 'Астана',
   'url': 'https://api.hh.ru/areas/159'},
  'salary': {'from': 110000, 'to': None, 'currency': 'KZT', 'gross': True},
  'salary_range': {'from': 110000,
   'to': None,
   'currency': 'KZT',
   'gross': True,
   'mode': {'id': 'MONTH', 'name': 'За\xa0месяц'},
   'frequency': {'id': 'MONTHLY', 'name': 'Раз в\xa0месяц'}},
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': None,
  'response_url': None,
  'sort_point_distance': None,
  'published_at': '2025-12-17T10:44:40+0300',
  'created_at': '2025-12-17T10:44:40+0300',
  'archived': False,
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=128818364',
  'show_contacts': True,
  'insider_interview': None,
  'url': 'https://api.hh.ru/vacancies/128818364?host=hh.ru',
  'alternate_url': 'https://hh.ru/vacancy/1288183

# Data scientist Job

In [58]:
import requests
import time

def get_vacancies():
    url = 'https://api.hh.ru/vacancies'
    jsons_list = []
    
    headers = {'User-Agent': 'TestBot'}
    
    # Data Scientist с хорошей зарплатой, удалённо
    params = {
        'text': 'Data Scientist',
        'per_page': '100',
        'page': 0,
        'only_with_salary': True,
        'schedule': 'remote',
        'salary': 150000,
        'currency': 'RUR',
        'search_field': 'name',
        'order_by': 'salary_desc',
        'experience': 'between3And6',
    }
    

    for num_page in range(5):
        params['page'] = num_page
        
        try:
            src = requests.get(url, params=params, headers=headers, timeout=10)
            
            if src.status_code != 200:
                print(f"Страница {num_page}: Ошибка {src.status_code}")
                break
                
            data = src.json()
            
            if 'items' not in data or not data['items']:
                print(f"Страница {num_page}: больше нет вакансий")
                break
                
            jsons_list.append(data)
            print(f"Страница {num_page+1}: {len(data['items'])} вакансий")
            
            # pause api to fix timeout
            time.sleep(0.3)
            
        except requests.exceptions.Timeout:
            print(f"Страница {num_page}: Таймаут, пропускаем")
            continue
        except Exception as e:
            print(f"Страница {num_page}: Ошибка {type(e).__name__}")
            break
    
    print(f"\nСобрано: {len(jsons_list)} страниц с вакансиями")
    if jsons_list:
        total_vacancies = sum(len(page['items']) for page in jsons_list)
        print(f"Всего вакансий: {total_vacancies}")
        
        # gold_job
        if jsons_list[0]['items']:
            top_vacancy = jsons_list[0]['items'][0]
            salary = top_vacancy.get('salary', {})
            print(f"gold_job: {top_vacancy['name']}")
            print(f"   coin: {salary.get('from', '?')} - {salary.get('to', '?')} {salary.get('currency', 'руб.')}")
            if 'schedule' in top_vacancy:
                print(f"   График: {top_vacancy['schedule'].get('name', '?')}")
    
    return jsons_list


In [59]:
vacancies = get_vacancies()

Страница 1: 10 вакансий
Страница 1: больше нет вакансий

Собрано: 1 страниц с вакансиями
Всего вакансий: 10
gold_job: Lead Data Scientist (LLM) / Ведущий специалист по данным
   coin: None - 5000 USD
   График: Удаленная работа


# Job Table

In [72]:
import pandas as pd

data = pd.DataFrame(columns=[
    'title',               # Название
    'salary_min',          # ЗП от
    'salary_max',          # ЗП до  
    'experience',          # Опыт 3-6
    'company',             # Компания
    'location',            # Локация
    'work_schedule',       # График = удаленка
    'remote_type',         # Формат
    'key_requirements',    # Ключевые требования
    'is_trusted_employer', # Проверенный работодатель
    'has_test',            # Тест!
    'url'                  # Ссылка
])

for col in data.columns:
    print(f"  -> {col}")

  -> title
  -> salary_min
  -> salary_max
  -> experience
  -> company
  -> location
  -> work_schedule
  -> remote_type
  -> key_requirements
  -> is_trusted_employer
  -> has_test
  -> url


In [73]:
for page in vacancies:
    if 'items' not in page:
        continue
        
    for vacancy in page['items']:
        row = {
            'title': vacancy.get('name', ''),
            'url': vacancy.get('alternate_url', ''),
        }
        
        salary_data = vacancy.get('salary')
        if salary_data:
            row['salary_min'] = salary_data.get('from')
            row['salary_max'] = salary_data.get('to')
        else:
            row['salary_min'] = None
            row['salary_max'] = None
        
        experience_data = vacancy.get('experience', {})
        row['experience'] = experience_data.get('name', '') if experience_data else ''
        
        employer_data = vacancy.get('employer', {})
        row['company'] = employer_data.get('name', '')
        row['is_trusted_employer'] = employer_data.get('trusted', False)
        
        area_data = vacancy.get('area', {})
        row['location'] = area_data.get('name', '')
        
        schedule_data = vacancy.get('schedule', {})
        row['work_schedule'] = schedule_data.get('name', '')
        
        work_format_data = vacancy.get('work_format', [{}])
        row['remote_type'] = work_format_data[0].get('name', '') if work_format_data else ''
        
        snippet_data = vacancy.get('snippet', {})
        requirements = snippet_data.get('requirement', '')
        row['key_requirements'] = requirements[:150] if requirements else ''
        
        row['has_test'] = vacancy.get('has_test', False)
        
        new_row_df = pd.DataFrame([row])
        data = pd.concat([data, new_row_df], ignore_index=True)

data

,title,salary_min,salary_max,experience,company,location,work_schedule,remote_type,key_requirements,is_trusted_employer,has_test,url
0,Lead Data Scientist (LLM) / Ведущий специалист...,None,5000,От 3 до 6 лет,WaveAccess,Санкт-Петербург,Удаленная работа,Удалённо,5+ years of commercial experience in <highligh...,True,False,https://hh.ru/vacancy/128483349
1,Senior ML / Data Scientist,3500,4350,От 3 до 6 лет,ПБК Менеджмент,Минск,Удаленная работа,Удалённо,Мы усиливаем команду разработки моделей и ищем...,True,False,https://hh.ru/vacancy/128152732
2,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Москва,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/127981401
3,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Казань,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913274
4,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Санкт-Петербург,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913275
5,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Екатеринбург,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913276
6,Data Scientist (senior),200000,400000,От 3 до 6 лет,Бюро судебного взыскания,Нижний Новгород,Удаленная работа,Удалённо,Знание и опыт работы с очередями и брокерами с...,True,False,https://hh.ru/vacancy/128913277
7,Data Scientist,None,300000,От 3 до 6 лет,Детский мир,Москва,Удаленная работа,На месте работодателя,Уверенное владение знаниями о работе и создани...,True,True,https://hh.ru/vacancy/127795453
8,Data analyst / Data scientist,230000,250000,От 3 до 6 лет,ROSSKO,Москва,Удаленная работа,Удалённо,Вы умеете выявлять факторы влияния на эффектив...,True,False,https://hh.ru/vacancy/127307468
9,Data Scientist,None,180000,От 3 до 6 лет,Hawking Bros,Москва,Удаленная работа,Удалённо,Опыт работы с табличными данными и классически...,True,False,https://hh.ru/vacancy/128102483


In [74]:
data.head(1)

,title,salary_min,salary_max,experience,company,location,work_schedule,remote_type,key_requirements,is_trusted_employer,has_test,url
0,Lead Data Scientist (LLM) / Ведущий специалист...,None,5000,От 3 до 6 лет,WaveAccess,Санкт-Петербург,Удаленная работа,Удалённо,5+ years of commercial experience in <highligh...,True,False,https://hh.ru/vacancy/128483349


Lead Data Scientist (LLM) / Ведущий специалист по данным

до 5 000 $ за месяц

Формат работы: удалённо

In [78]:
data.iloc[0]['key_requirements']

'5+ years of commercial experience in <highlighttext>data</highlighttext> science or ML engineering. Ability to design the overall approach to solving.'

Виджет по api.hh.ru

In [ ]:
from IPython.display import IFrame

# url
widget_url = "https://api.hh.ru/widgets/vacancies/employer?employer_id=2251053&locale=RU&links_color=1560b2&border_color=1560b2&title=&show_region=true&professional_role=165"

# widget
IFrame(src=widget_url, width=350, height=250)